# Config

In [1]:
file_path = "/content/drive/MyDrive/mimic-iii-clinicalnote-v4.1/bq-results-20231215-074521-1702626519432.csv"
output_folder_path = "/content/drive/MyDrive/mimic-iii-clinicalnote-v4.1/processed_set"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# import package and load the data

In [3]:
import pandas as pd
import numpy as np
import torch
import os
import string
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel

# load data

In [4]:
X_train = pd.read_csv(os.path.join(output_folder_path, 'X_train_raw.csv'))
X_test = pd.read_csv(os.path.join(output_folder_path, 'X_test_raw.csv'))
X_val = pd.read_csv(os.path.join(output_folder_path, 'X_val_raw.csv'))

X_train

,icustay_id,slice_start,avg_pCO2,avg_Glucose,avg_Platelet_Count,avg_Potassium,avg_Calculated_total_CO2,avg_Urea_nitrogen,avg_Respiratory_rate,avg_Heartrate,avg_Arterial_blood_pressure_mean,avg_Arterial_blood_pressure_systolic,avg_Central_venous_pressure,avg_Arterial_blood_pressure_diastolic,avg_Tidal_volume_obs,avg_Temp_C,avg_Height_cm,processed_text
0,200006,2159-09-03 11:28:14,-0.003207,0.053683,-0.099660,-1.073710,-0.054551,-0.39023,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,missing
1,200006,2159-09-03 13:28:14,-0.003207,0.053683,-0.099660,-1.073710,-0.054551,-0.39023,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,missing
2,200006,2159-09-03 15:28:14,-0.003207,0.053683,-0.099660,-1.073710,-0.054551,-0.39023,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,missing
3,200006,2159-09-03 17:28:14,-0.003207,0.053683,-0.099660,-1.073710,-0.054551,-0.39023,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,hospital ward name 4 icu nursing admitprogress...
4,200006,2159-09-03 19:28:14,-0.003207,0.053683,-0.099660,-1.073710,-0.054551,-0.39023,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,patient admitted hospital ward name 6 ett stom...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248791,299995,2116-03-05 07:44:39,-0.068601,-0.442837,0.186447,-0.978583,0.199645,-0.92366,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,respiratory care pt extubated today 0745 post ...
248792,299995,2116-03-05 09:44:39,-0.068601,-0.442837,0.186447,-0.978583,0.199645,-0.92366,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,missing
248793,299995,2116-03-05 11:44:39,-0.068601,-0.442837,0.186447,-0.978583,0.199645,-0.92366,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,missing
248794,299995,2116-03-05 13:44:39,-0.068601,-0.442837,0.186447,-0.978583,0.199645,-0.92366,0.003611,0.00263,0.005481,0.004993,-0.00816,-0.000732,-0.010367,0.025811,0.000348,missing


In [5]:
X_train_clinicalbert = X_train[['icustay_id', 'slice_start', 'processed_text']]
X_test_clinicalbert = X_test[['icustay_id', 'slice_start', 'processed_text']]
X_val_clinicalbert = X_val[['icustay_id', 'slice_start', 'processed_text']]
X_train_clinicalbert

,icustay_id,slice_start,processed_text
0,200006,2159-09-03 11:28:14,missing
1,200006,2159-09-03 13:28:14,missing
2,200006,2159-09-03 15:28:14,missing
3,200006,2159-09-03 17:28:14,hospital ward name 4 icu nursing admitprogress...
4,200006,2159-09-03 19:28:14,patient admitted hospital ward name 6 ett stom...
...,...,...,...
248791,299995,2116-03-05 07:44:39,respiratory care pt extubated today 0745 post ...
248792,299995,2116-03-05 09:44:39,missing
248793,299995,2116-03-05 11:44:39,missing
248794,299995,2116-03-05 13:44:39,missing


In [6]:
X_train_clinicalbert['processed_text'] = X_train_clinicalbert['processed_text'].fillna('missing')
X_test_clinicalbert['processed_text'] = X_test_clinicalbert['processed_text'].fillna('missing')
X_val_clinicalbert['processed_text'] = X_val_clinicalbert['processed_text'].fillna('missing')
X_train_clinicalbert

<ipython-input-6-3a3c11313417>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_clinicalbert['processed_text'] = X_train_clinicalbert['processed_text'].fillna('missing')
<ipython-input-6-3a3c11313417>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_clinicalbert['processed_text'] = X_test_clinicalbert['processed_text'].fillna('missing')
<ipython-input-6-3a3c11313417>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

,icustay_id,slice_start,processed_text
0,200006,2159-09-03 11:28:14,missing
1,200006,2159-09-03 13:28:14,missing
2,200006,2159-09-03 15:28:14,missing
3,200006,2159-09-03 17:28:14,hospital ward name 4 icu nursing admitprogress...
4,200006,2159-09-03 19:28:14,patient admitted hospital ward name 6 ett stom...
...,...,...,...
248791,299995,2116-03-05 07:44:39,respiratory care pt extubated today 0745 post ...
248792,299995,2116-03-05 09:44:39,missing
248793,299995,2116-03-05 11:44:39,missing
248794,299995,2116-03-05 13:44:39,missing


# ClinicalBERT embedding

In [7]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("You're using: ", device)
# Load pre-trained Clinical BERT model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
Bio_ClinicalBERT_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Send the model to GPU if available
Bio_ClinicalBERT_model = Bio_ClinicalBERT_model.to(device)

# Function to generate embeddings with a progress bar
def get_clinical_bert_embeddings(texts):
    embeddings = []
    for text in tqdm(texts, desc="Generating Embeddings"):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = Bio_ClinicalBERT_model(**inputs)
        # Use the average of the last hidden state for sentence embedding
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(embedding.flatten().tolist())
    return embeddings

You're using:  cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
X_train_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddings(X_train_clinicalbert['processed_text'])
X_test_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddings(X_test_clinicalbert['processed_text'])
X_val_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddings(X_val_clinicalbert['processed_text'])

Generating Embeddings: 100%|██████████| 248796/248796 [46:14<00:00, 89.67it/s]
<ipython-input-8-96357def0ddf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddings(X_train_clinicalbert['processed_text'])
Generating Embeddings: 100%|██████████| 82944/82944 [15:21<00:00, 90.02it/s]
<ipython-input-8-96357def0ddf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddin

In [ ]:
# Save the datasets
X_train_clinicalbert.to_csv(os.path.join(output_folder_path, 'X_train_clinicalbert.csv'), index=False)
X_test_clinicalbert.to_csv(os.path.join(output_folder_path, 'X_test_clinicalbert.csv'), index=False)
X_val_clinicalbert.to_csv(os.path.join(output_folder_path, 'X_val_clinicalbert.csv'), index=False)